In [ ]:
import os
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
# then set root = '/gdrive/My Drive'
root = '/gdrive/My Drive'

In [ ]:
import numpy as np
import torch
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
import torchvision
import pandas as pd

In [ ]:
gpu = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
gpu

device(type='cpu')

In [ ]:
print("PyTorch version:", torch.__version__)
print("CUDA version:", torch.version.cuda)
print("cuDNN version:", torch.backends.cudnn.version())

PyTorch version: 1.7.0+cu101
CUDA version: 10.1
cuDNN version: 7603


In [ ]:
class Linear_Model(nn.Module):
    def __init__(self,num_input):
        super(Linear_Model,self).__init__()

        self.inputNum = num_input
        self.fc = nn.Linear(num_input,1)
    
    def forward(self,x):
        return self.fc(x)




In [ ]:
cor_file_name = root + "/Code/linear_regression/tr_cor.csv"
incor_file_name = root + "/Code/linear_regression/tr_incor.csv"

cor_data = pd.read_csv(cor_file_name)
incor_data = pd.read_csv(incor_file_name)

cor_list = np.array(cor_data).tolist()
incor_list = np.array(incor_data).tolist()

new_cor_list = []
for i in range(len(cor_list[0])-1):
    new_cor_list.append([])

for i in range(len(cor_list)):
    for j in range(1,len(cor_list[i])):
        new_cor_list[j-1].append(cor_list[i][j])

train_cor_list = new_cor_list[:int(len(new_cor_list)*0.8)]
test_cor_list = new_cor_list[int(len(new_cor_list)*0.8):]

train_cor = torch.tensor(train_cor_list)
test_cor = torch.tensor(test_cor_list)

new_incor_list = []
for i in range(len(incor_list[0])-1):
    new_incor_list.append([])

for i in range(len(incor_list)):
    for j in range(1,len(incor_list[i])):
        new_incor_list[j-1].append(incor_list[i][j])

train_incor_list = new_incor_list[:int(len(new_incor_list)*0.8)]
test_incor_list = new_incor_list[int(len(new_incor_list)*0.8):]

train_incor = torch.tensor(train_incor_list)
test_incor = torch.tensor(test_incor_list)

In [ ]:
train_data = torch.cat((train_cor,train_incor))
print(train_data.size())
print(train_data)

print(test_cor.size())
print(test_incor.size())

torch.Size([1170, 9])
tensor([[172.4000, 215.2000, 242.2000,  ...,  94.8333, 165.7500, 204.0000],
        [115.4000, 185.4000, 222.0000,  ..., 121.5000, 182.0000, 211.5000],
        [110.2000, 180.8000, 218.0000,  ..., 105.5000, 166.8333, 194.5000],
        ...,
        [111.8000,  80.2000,  67.0000,  ..., 192.7500, 179.2500, 178.9167],
        [171.8000, 123.8000, 119.8000,  ..., 152.8333, 200.8333, 227.2500],
        [115.0000,  72.6000,  55.6000,  ..., 129.0833, 194.5833, 224.6667]])
torch.Size([129, 9])
torch.Size([164, 9])


In [ ]:
cor_label = torch.ones(train_cor.size(0)).unsqueeze(-1)
incor_label = torch.zeros(train_incor.size(0)).unsqueeze(-1)
label = torch.cat((cor_label,incor_label))
print(label.size())
print(label)

torch.Size([1170, 1])
tensor([[1.],
        [1.],
        [1.],
        ...,
        [0.],
        [0.],
        [0.]])


In [ ]:
model = Linear_Model(train_data.size(1))

print(list(model.parameters()))

optimizer = torch.optim.SGD(model.parameters(),lr=1e-6)

[Parameter containing:
tensor([[ 0.2892,  0.1770,  0.3085, -0.1651,  0.1140,  0.2993,  0.0208, -0.2858,
          0.0135]], requires_grad=True), Parameter containing:
tensor([-0.0341], requires_grad=True)]


In [ ]:
epochs = 100000
for epoch in range(epochs+1):
    prediction = model(train_data)

    loss = F.mse_loss(prediction,label)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print("epoch {:4d}, loss {:.6f}".format(epoch,loss.item()))

epoch    0, loss 17961.125000
epoch  100, loss 98.839218
epoch  200, loss 47.408257
epoch  300, loss 35.596222
epoch  400, loss 27.841122
epoch  500, loss 22.127222
epoch  600, loss 17.835596
epoch  700, loss 14.578421
epoch  800, loss 12.086463
epoch  900, loss 10.166211
epoch 1000, loss 8.675880
epoch 1100, loss 7.510453
epoch 1200, loss 6.591630
epoch 1300, loss 5.860763
epoch 1400, loss 5.273787
epoch 1500, loss 4.797501
epoch 1600, loss 4.406815
epoch 1700, loss 4.082712
epoch 1800, loss 3.810732
epoch 1900, loss 3.579838
epoch 2000, loss 3.381572
epoch 2100, loss 3.209424
epoch 2200, loss 3.058362
epoch 2300, loss 2.924468
epoch 2400, loss 2.804681
epoch 2500, loss 2.696588
epoch 2600, loss 2.598277
epoch 2700, loss 2.508224
epoch 2800, loss 2.425200
epoch 2900, loss 2.348211
epoch 3000, loss 2.276448
epoch 3100, loss 2.209246
epoch 3200, loss 2.146050
epoch 3300, loss 2.086402
epoch 3400, loss 2.029912
epoch 3500, loss 1.976254
epoch 3600, loss 1.925148
epoch 3700, loss 1.876355

In [ ]:
answer_cor = 0
wrong_cor = 0

for i in range(test_cor.size(0)):
    pred = model(test_cor[i])

    if pred[0] >= 0.4:
        answer_cor += 1
    
    if pred[0] < 0.2:
        wrong_cor += 1
    
print("correct set {}/{}      -{}".format(answer_cor,test_cor.size(0),wrong_cor))


answer_incor = 0
wrong_incor = 0

for i in range(test_incor.size(0)):
    pred = model(test_incor[i])

    if pred[0] < 0.4:
        answer_incor += 1

    if pred[0] >= 0.8:
        wrong_cor += 1
    
print("incorrect set {}/{}      -{}".format(answer_incor,test_incor.size(0),wrong_incor))


accuracy = (answer_cor+answer_incor)/(test_cor.size(0)+test_incor.size(0))*100

print("Accuracy: {}".format(accuracy))


correct set 127/129      -2
incorrect set 155/164      -0
Accuracy: 96.24573378839591


In [ ]:
print(list(model.parameters()))

model_path = root + "/Code/linear_regression/mask_correctness.pt"
torch.save(model,model_path)

[Parameter containing:
tensor([[-0.0054, -0.0001,  0.0065, -0.0126,  0.0538, -0.0403,  0.0179, -0.0712,
          0.0532]], requires_grad=True), Parameter containing:
tensor([-0.0362], requires_grad=True)]


In [ ]:
new_model = torch.load(model_path)
print(list(new_model.parameters()))
new_model.eval()
print(list(new_model.parameters()))


[Parameter containing:
tensor([[-0.0054, -0.0001,  0.0065, -0.0126,  0.0538, -0.0403,  0.0179, -0.0712,
          0.0532]], requires_grad=True), Parameter containing:
tensor([-0.0362], requires_grad=True)]
[Parameter containing:
tensor([[-0.0054, -0.0001,  0.0065, -0.0126,  0.0538, -0.0403,  0.0179, -0.0712,
          0.0532]], requires_grad=True), Parameter containing:
tensor([-0.0362], requires_grad=True)]


In [ ]:
def classify_mask_corr(model,image):
    
    pred = model(rgb_tensor)
    if pred < 0.5:
        return 0
    else:
        return 1